Order 
=====

Einfacher Microserives welcher neue Bestellungen (Orders) mittels HTTP POST entgegenimmt und diese anzeigt.

In [ ]:
from flask import Flask, request, jsonify, render_template_string
import socket
import os
import requests
import time
from collections import defaultdict

app = Flask(__name__)

In [ ]:
# Mockup Daten
products = [
    {"id": 1, "name": "Land Rover Range Rover", "price": 150000},
    {"id": 2, "name": "BMW 3er", "price": 60000},
    {"id": 3, "name": "Audi A4", "price": 60000},
    {"id": 4, "name": "Audi e-tron", "price": 100000},
    {"id": 5, "name": "Mercedes-Benz C-Klasse", "price": 70000},
    {"id": 6, "name": "Toyota Corolla", "price": 35000},
    {"id": 7, "name": "Tesla Model 3", "price": 55000},
    {"id": 8, "name": "Ford Mustang", "price": 60000},
    {"id": 9, "name": "Volkswagen ID.4", "price": 60000}
]

customers = [
    {"id": 1, "name": "John Doe", "email": "john.doe@example.com"},
    {"id": 2, "name": "Jane Smith", "email": "jane.smith@example.com"},
    {"id": 3, "name": "Bob Johnson", "email": "bob.johnson@example.com"}
]

orders = [
    {"id": 1, "product_id": 1, "quantity": 1, "customer_id": 1},
    {"id": 2, "product_id": 4, "quantity": 2, "customer_id": 2},
    {"id": 3, "product_id": 8, "quantity": 1, "customer_id": 3}
]

In [ ]:
# Funktionen, um die Daten von internen Tabellen abzurufen
def get_orders():
    return orders

def get_customers():
    return customers

def get_catalog():
    return products

In [ ]:
# Funktion, um die Daten für die Tabellen aufzubereiten
def process_data(orders, customers, catalog):
    customers_dict = {customer["id"]: customer for customer in customers}
    catalog_dict = {item["id"]: item for item in catalog}
    
    invoices = defaultdict(lambda: {"customer_name": "", "total_amount": 0.0, "entries": []})
    for order in orders:
        customer = customers_dict.get(order["customer_id"], {})
        product = catalog_dict.get(order["product_id"], {})
        
        if customer and product:
            total = order["quantity"] * product["price"]
            invoices[order["customer_id"]]["customer_name"] = customer["name"]
            invoices[order["customer_id"]]["total_amount"] += total
            invoices[order["customer_id"]]["entries"].append({
                "order_id": order["id"],
                "product_name": product["name"],
                "quantity": order["quantity"],
                "product_price": product["price"],
                "total": total
            })
    return invoices

In [ ]:
# Ausgabe der Daten im HTML Format
@app.route('/', methods=['GET'])
def get_orders_html():
    start_time = time.time()
    try:
        orders = get_orders()
        customers = get_customers()
        catalog = get_catalog()
        
        invoices = process_data(orders, customers, catalog)
        response_data = [{"customer_id": customer_id, "customer_name": details["customer_name"], "total_amount": details["total_amount"], "entries": details["entries"]} for customer_id, details in invoices.items()]

        hostname = socket.gethostname()
        total_all = sum(details["total_amount"] for details in invoices.values())
        html = """
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
            <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
            <title>Order Service on {{ hostname }}</title>
        </head>
        <body>
            <div class="container">
                <h1 class="mt-5">Order Service on {{ hostname }}</h1>
                <table class="table table-striped mt-3">
                    <thead>
                        <tr>
                            <th>Customer ID</th>
                            <th>Customer Name</th>
                            <th>Order ID</th>
                            <th>Product Name</th>
                            <th>Quantity</th>
                            <th>Product Price</th>
                            <th>Total</th>
                        </tr>
                    </thead>
                    <tbody>
                        {% for entry in data %}
                            {% for order in entry.entries %}
                            <tr>
                                <td>{{ entry.customer_id }}</td>
                                <td>{{ entry.customer_name }}</td>
                                <td>{{ order.order_id }}</td>
                                <td>{{ order.product_name }}</td>
                                <td>{{ order.quantity }}</td>
                                <td>${{ order.product_price }}</td>
                                <td>${{ order.total }}</td>
                            </tr>
                            {% endfor %}
                            <tr>
                                <td colspan="6"><strong>Total for {{ entry.customer_name }}</strong></td>
                                <td><strong>${{ entry.total_amount }}</strong></td>
                            </tr>
                        {% endfor %}
                        <tr>
                            <td colspan="6"><strong>Total</strong></td>
                            <td><strong>${{ total_all }}</strong></td>
                        </tr>
                    </tbody>
                </table>
            </div>
        </body>
        </html>
        """
        response = render_template_string(html, data=response_data, hostname=hostname, total_all=total_all)
        status = 200
    except Exception as e:
        response = jsonify({"status": "error", "message": str(e)})
        status = 500
    return response, status


In [ ]:
# Hinzufügen von Daten mittels HTTP POST
@app.route('/', methods=['POST'])
def add_order():
    try:
        data = request.get_json()
        if not data or not all(key in data for key in ("product_id", "quantity", "customer_id")):
            return jsonify({"status": "error", "message": "Invalid request"}), 400

        new_order_id = max(order['id'] for order in orders) + 1 if orders else 1
        new_order = {
            "id": new_order_id,
            "product_id": data["product_id"],
            "quantity": data["quantity"],
            "customer_id": data["customer_id"]
        }
        orders.append(new_order)
        print(data)  # Daten auf der Konsole ausgeben
        return jsonify({"status": "success", "data": new_order}), 201
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500

### Main

Anzeige der Bestellung mittels des untenstehenden URLs.

Hinzufügen von weiteren Bestellungen mittels `curl`, wie unten angezeigt.

In [ ]:
! echo "http://$(cat ~/work/server-ip):8080"
! echo "curl -X POST http://$(cat ~/work/server-ip):8080 -H 'Content-Type: application/json' -d '{ \"product_id\": 1, \"quantity\": 1, \"customer_id\": 1}'"

In [ ]:
if __name__ == '__main__':
    # Start up the server
    app.run(host='0.0.0.0', port=8080)